# <center> Itinerary Network Problem for two periods </center>

## Part (a) Normal economy class in the first period {60,30} prior to departure.
### 1. Dual solution for Deterministic LP

In [1]:
from pyutilib.services import register_executable, registered_executable
register_executable(name='gurobi')
import numpy as np
import itertools
from pyomo.environ import *
import pyomo.environ as pyo
from pyomo.core import *
from pyomo.opt import SolverFactory
from pyomo.pysp.annotations import (PySP_ConstraintStageAnnotation, StochasticConstraintBoundsAnnotation, StochasticConstraintBodyAnnotation)

In [3]:
model = ConcreteModel()
opt = SolverFactory('gurobi_ampl')

X = ['x3','x4','x5']
model.x = Var(X,within=NonNegativeIntegers)
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT_EXPORT)

v = {1:280,2:300,3:190,4:220,5:140}
c = [[0,1,1,0,0,1],
     [1,0,1,1,0,0],
     [0,1,1,0,0,1],
     [1,0,1,1,0,0],
     [0,0,1,0,1,0]]
capa = {1:100,2:100,3:300,4:100,5:100,6:100}
d_mean = {1:50,2:30,3:200,4:100,5:160}

# Objective 
model.obj = Objective(expr= model.x['x3']*v[3]+model.x['x4']*v[4]+model.x['x5']*v[5],sense=maximize)

# Capacity Constraint
c1 =np.transpose(c)[0]
model.c1 = Constraint(expr= model.x['x3']*c1[2]
                      +model.x['x4']*c1[3]+model.x['x5']*c1[4] <= capa[1])
c2 =np.transpose(c)[1]
model.c2 = Constraint(expr= model.x['x3']*c2[2]
                      +model.x['x4']*c2[3]+model.x['x5']*c2[4] <= capa[2])
c3 =np.transpose(c)[2]
model.c3 = Constraint(expr= model.x['x3']*c3[2]
                      +model.x['x4']*c3[3]+model.x['x5']*c3[4] <= capa[3])
c4 =np.transpose(c)[3]
model.c4 = Constraint(expr= model.x['x3']*c4[2]
                      +model.x['x4']*c4[3]+model.x['x5']*c4[4] <= capa[4])
c5 =np.transpose(c)[4]
model.c5 = Constraint(expr= model.x['x3']*c5[2]
                      +model.x['x4']*c5[3]+model.x['x5']*c5[4] <= capa[5])
c6 =np.transpose(c)[5]
model.c6 = Constraint(expr= model.x['x3']*c6[2]
                      +model.x['x4']*c6[3]+model.x['x5']*c6[4] <= capa[6])

# Remaining Demand Constraint (when D is average D)

upper = {'x1':50, 'x2':30, 'x3':200, 'x4':100, 'x5':160}
 
model.d3 = Constraint(expr = model.x['x3'] <= upper['x3']) 
model.d4 = Constraint(expr = model.x['x4'] <= upper['x4']) 
model.d5 = Constraint(expr = model.x['x5'] <= upper['x5']) 

In [4]:
results = opt.solve(model,tee=True)
model.display()
model.dual.display()

Academic license - for non-commercial use only
Gurobi 8.1.0Model unknown

  Variables:
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
         x3 :     0 : 100.0 :  None : False : False : NonNegativeIntegers
         x4 :     0 : 100.0 :  None : False : False : NonNegativeIntegers
         x5 :     0 : 100.0 :  None : False : False : NonNegativeIntegers

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 55000.0

  Constraints:
    c1 : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 100.0 : 100.0
    c2 : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 100.0 : 100.0
    c3 : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 300.0 : 300.0
    c4 : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 100.0 : 100.0
    c5 : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 100.0 : 100.0
    c6 : Size=1

### 2. Dual solution for PNLP

In [5]:
# DLP with 2 periods, period 1
import numpy as np
import pandas as pd
import itertools
from pyomo.environ import *
from pyomo.core import *
from pyomo.pysp.annotations import (PySP_ConstraintStageAnnotation, StochasticConstraintBoundsAnnotation, 
                                    StochasticConstraintBodyAnnotation)

model = ConcreteModel()

v = {1:280,2:300,3:190,4:220,5:140}
c = [[0,1,1,0,0,1],
     [1,0,1,1,0,0],
     [0,1,1,0,0,1],
     [1,0,1,1,0,0],
     [0,0,1,0,1,0]]
capa = {1:100,2:100,3:300,4:100,5:100,6:100}
d_mean = {1:50,2:30,3:200,4:100,5:160}

# Possible Demands 
d1_rhs_table = [25,50,75]
d2_rhs_table = [15,30,45]
d3_rhs_table = [100,200,300]
d4_rhs_table = [50,100,150]
d5_rhs_table = [80,160,240]

model.constraint_stage = PySP_ConstraintStageAnnotation()
model.stoch_rhs = StochasticConstraintBoundsAnnotation()
num_scenarios = len(d3_rhs_table) * len(d4_rhs_table) * len(d5_rhs_table)
scenario_data = dict(('Scenario'+str(i), (d3val, d4val, d5val))
                     for i, (d3val, d4val, d5val) in
                     enumerate (itertools.product(d3_rhs_table, d4_rhs_table, d5_rhs_table),1))

# Declare Variables
# model.x1 = Var(within=NonNegativeIntegers)
# model.x2 = Var(within=NonNegativeIntegers)
model.x3 = Var(within=NonNegativeIntegers)
model.x4 = Var(within=NonNegativeIntegers)
model.x5 = Var(within=NonNegativeIntegers)

# model.t1 = Var(within=NonNegativeIntegers)
# model.t2 = Var(within=NonNegativeIntegers)
model.t3 = Var(within=NonNegativeIntegers)
model.t4 = Var(within=NonNegativeIntegers)
model.t5 = Var(within=NonNegativeIntegers)

# model.d1_rhs = Param(mutable=True, initialize=0.0)
# model.d2_rhs = Param(mutable=True, initialize=0.0)
model.d3_rhs = Param(mutable=True, initialize=0.0)
model.d4_rhs = Param(mutable=True, initialize=0.0)
model.d5_rhs = Param(mutable=True, initialize=0.0)

# Objective 
# model.obj = Objective(expr= model.x['x1']*v[1]+model.x['x2']*v[2]+model.x['x3']*v[3]
#                       +model.x['x4']*v[4]+model.x['x5']*v[5],sense=maximize)
model.FirstStageCost = Expression(initialize=0)
model.SecondStageCost = Expression(initialize=model.t3*v[3]+model.t4*v[4]+model.t5*v[5])

# Capacity Constraints
c1 =np.transpose(c)[0]
model.c1 = Constraint(expr= model.x3*c1[2]
                      +model.x4*c1[3]+model.x5*c1[4] <= capa[1])
model.constraint_stage.declare(model.c1,1)
c2 =np.transpose(c)[1]
model.c2 = Constraint(expr= model.x3*c2[2]
                      +model.x4*c2[3]+model.x5*c2[4] <= capa[2])
model.constraint_stage.declare(model.c2,1)
c3 =np.transpose(c)[2]
model.c3 = Constraint(expr= model.x3*c3[2]
                      +model.x4*c3[3]+model.x5*c3[4] <= capa[3])
model.constraint_stage.declare(model.c3,1)
c4 =np.transpose(c)[3]
model.c4 = Constraint(expr= model.x3*c4[2]
                      +model.x4*c4[3]+model.x5*c4[4] <= capa[4])
model.constraint_stage.declare(model.c4,1)
c5 =np.transpose(c)[4]
model.c5 = Constraint(expr= model.x3*c5[2]
                      +model.x4*c5[3]+model.x5*c5[4] <= capa[5])
model.constraint_stage.declare(model.c5,1)
c6 =np.transpose(c)[5]
model.c6 = Constraint(expr= model.x3*c6[2]
                      +model.x4*c6[3]+model.x5*c6[4] <= capa[6])
model.constraint_stage.declare(model.c6,1)

# Second Stage Min{X,D} Constraint (when D is stochastic)
# model.d11 = Constraint(expr=model.t1 <= model.d1_rhs)
# model.constraint_stage.declare(model.d11,2)
# model.stoch_rhs.declare(model.d11)

# model.d12 = Constraint(expr=model.t1 - model.x1 <= 0)
# model.constraint_stage.declare(model.d12,2)

# model.d21 = Constraint(expr=model.t2 <= model.d2_rhs)
# model.constraint_stage.declare(model.d21,2)
# model.stoch_rhs.declare(model.d21)

# model.d22 = Constraint(expr=model.t2 - model.x2 <= 0)
# model.constraint_stage.declare(model.d22,2)

model.d31 = Constraint(expr=model.t3 <= model.d3_rhs)
model.constraint_stage.declare(model.d31,2)
model.stoch_rhs.declare(model.d31)

model.d32 = Constraint(expr=model.t3 - model.x3 <= 0)
model.constraint_stage.declare(model.d32,2)

model.d41 = Constraint(expr=model.t4 <= model.d4_rhs)
model.constraint_stage.declare(model.d41,2)
model.stoch_rhs.declare(model.d41)

model.d42 = Constraint(expr=model.t4 - model.x4 <= 0)
model.constraint_stage.declare(model.d42,2)

model.d51 = Constraint(expr=model.t5 <= model.d5_rhs)
model.constraint_stage.declare(model.d51,2)
model.stoch_rhs.declare(model.d51)

model.d52 = Constraint(expr=model.t5 - model.x5 <= 0)
model.constraint_stage.declare(model.d52,2)

# Final Objective
model.obj = Objective(expr=(model.FirstStageCost+model.SecondStageCost)*(-1))

    (constraint stages are automatically inferred). It will be removed in the
    future. Please update your model file.


In [6]:
def pysp_scenario_tree_model_callback():
    from pyomo.pysp.scenariotree.tree_structure_model import \
        CreateConcreteTwoStageScenarioTreeModel

    st_model = CreateConcreteTwoStageScenarioTreeModel(num_scenarios)

    first_stage = st_model.Stages.first()
    second_stage = st_model.Stages.last()

    # First Stage
    st_model.StageCost[first_stage] = 'FirstStageCost'  
#     st_model.StageVariables[first_stage].add('x1')
#     st_model.StageVariables[first_stage].add('x2')
    st_model.StageVariables[first_stage].add('x3')
    st_model.StageVariables[first_stage].add('x4')
    st_model.StageVariables[first_stage].add('x5')
    
    # Second Stage
    st_model.StageCost[second_stage] = 'SecondStageCost'
#     st_model.StageVariables[second_stage].add('t1')
#     st_model.StageVariables[second_stage].add('t2')
    st_model.StageVariables[second_stage].add('t3')
    st_model.StageVariables[second_stage].add('t4')
    st_model.StageVariables[second_stage].add('t5')
    return st_model

def pysp_instance_creation_callback(scenario_name, node_names):

    instance = model.clone()

    d3_rhs_val, d4_rhs_val, d5_rhs_val  = scenario_data[scenario_name]
#     instance.d1_rhs.value = d1_rhs_val
#     instance.d2_rhs.value = d2_rhs_val
    instance.d3_rhs.value = d3_rhs_val
    instance.d4_rhs.value = d4_rhs_val
    instance.d5_rhs.value = d5_rhs_val

    return instance

In [6]:
!python -m pyomo.pysp.convert.smps -m itinerary_network_2class.py --basename Itinerary \--output-directory sdinput/Itinerary --symbolic-solver-labels

/anaconda3/envs/yulenv/lib/python3.6/runpy.py:125: RuntimeWarning: 'pyomo.pysp.convert.smps' found in sys.modules after import of package 'pyomo.pysp.convert', but prior to execution of 'pyomo.pysp.convert.smps'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
ERROR: Failed to extract reference model python file from path specification:
    itinerary_network_2class.py
pyomo.pysp.convert.smps: IO ERROR:
path does not exist: /Users/yul/Desktop/Github Fetch/Pyomo/itinerary_network_2class.py

To obtain further information regarding the source of the exception, use the --traceback option


Problem:                                itinerary
First Stage Rows:                       7
First Stage Columns:                    3
First Stage Non-zeros:                  8
Mean solution is recommended for this instance.
Number of replications:                 3
Status:                                 MEAN SOLUTION
Total Objective Function Upper Bound:   -50290.609,[-50541.940188,-50039.277797],half-width:251.331, stdev:128.230
Total Objective Function Lower Bound:   -50557.112,[-50663.181388,-50451.041899],half-width:106.070, stdev:24.835

First Stage Solutions:
   No.   Row name   Activity      Lower bound   Upper bound   Dual          Dual STDEV
------ ------------ ------------- ------------- ------------- ------------- -------------
     0 obj          -5.029061e+04 
     1 c_u_c1_       1.000000e+02                1.000000e+02  0.000000e+00  0.000000e+00 
     2 c_u_c2_       1.000000e+02                1.000000e+02 -1.787758e+02  1.691334e+01 
     3 c_u_c3_       3.000000e+02                3.000000e+02  0.000000e+00  0.000000e+00 
     4 c_u_c4_       1.000000e+02                1.000000e+02 -9.428991e+01  2.155608e+01 
     5 c_u_c5_       1.000000e+02                1.000000e+02 -8.950978e+01  4.219948e+00 
     6 c_u_c6_       1.000000e+02                1.000000e+02  0.000000e+00  0.000000e+00 

   No. Column name  Activity      Lower bound   Upper bound   Reduced Cost  RC STDEV
------ ------------ ------------- ------------- ------------- ------------- -------------
     1 x3            1.000000e+02  0.000000e+00               0.000000e+00  0.000000e+00 
     2 x4            1.000000e+02  0.000000e+00               0.000000e+00  0.000000e+00 
     3 x5            1.000000e+02  0.000000e+00               0.000000e+00  0.000000e+00 


### Generate 25 replications with random seeds deriven from seed(1) and calculate the expected total revenue with respect to the dual price for each itinerary. <br>
#### The customer bids of period 1 is driven from the uniform distribution $ \{v_i-0.3v_i,  v_i+0.3v_i\} $
#### 1. DLP :  Calculate the 95% Confidence Interval for expected total revenue.

In [3]:
np.random.seed(1)
seed_lst= np.random.randint(100,size=25)

# General Form for 5 bids
v = [280,300,190,220,140]
v_dual = [0,0,190,0,140] 

v_bid = []    
for i in range(len(seed_lst)): # create a list w.r.t. seeds with nested v_i lists
    np.random.seed(seed_lst[i])
    v_bid.append([])
    for j in range(5):
        v_bid[i].append(np.random.randint(low=v[j]*0.7,high=v[j]*1.3)) # fills nested lists with data
        
for i in range(len(seed_lst)):
    for j in range(5):
        if v_bid[i][j] >= v_dual[j]:
            v_bid[i][j] = 0
        else: v_bid[i][j]
# print(v_bid)

# For revenue calculation of itinerary 3,4,5, this will erase itinerary 1,2
for i in range(len(seed_lst)):
    for j in range(2):
        v_bid[i][j] = 0
# print(v_bid)

# Total revenue for 25 replications
total_revenue=[]
for i in range(len(seed_lst)):
    total_revenue.append(v_bid[i][0]*0+v_bid[i][1]*0+v_bid[i][2]*102
                        +v_bid[i][3]*26+v_bid[i][4]*58)
# print(total_revenue)

import scipy.stats as st
ci1 = st.t.interval(0.95, len(total_revenue)-1, loc=np.mean(total_revenue),
              scale=st.sem(total_revenue))
print(ci1)

# number of passengers who bought during the first period
d_origin = [0,0,102,26,58]
d_new1 = []   
for i in range(len(seed_lst)):
    d_new1.append([])
    for j in range(5):
        d_new1[i].append(d_origin[j] if v_bid[i][j]>0 else 0)

(8099.3010577808809, 16166.458942219117)


#### 2. PNLP :  Calculate the 95% Confidence Interval for expected total revenue.

In [31]:
np.random.seed(1)
seed_lst= np.random.randint(100,size=25)

# General Form for 5 bids
v = [280,300,190,220,140]
v_dual = [0,0,178.78,94.29,89.51]

v_bid = []    
for i in range(len(seed_lst)): # create a list w.r.t. seeds with nested v_i lists
    np.random.seed(seed_lst[i])
    v_bid.append([])
    for j in range(5):
        v_bid[i].append(np.random.randint(low=v[j]*0.7,high=v[j]*1.3)) # fills nested lists with data
# print(v_bid)

for i in range(len(seed_lst)):
    for j in range(5):
        if v_bid[i][j] >= v_dual[j]:
            v_bid[i][j] = 0
        else: v_bid[i][j]
# print(v_bid)

# For revenue calculation of itinerary 3,4,5, this will erase itinerary 1,2
for i in range(len(seed_lst)):
    for j in range(2):
        v_bid[i][j] = 0
# print(v_bid)

# Total revenue for 25 replications
total_revenue=[]
for i in range(len(seed_lst)):
    total_revenue.append(v_bid[i][0]*0+v_bid[i][1]*0+v_bid[i][2]*102
                        +v_bid[i][3]*26+v_bid[i][4]*58)
# print(total_revenue)

import scipy.stats as st
ci2 = st.t.interval(0.95, len(total_revenue)-1, loc=np.mean(total_revenue),
              scale=st.sem(total_revenue))
print(ci2)

# number of passengers who bought during the first period.
d_origin = [0,0,102,26,58]
d_new2 = []   
for i in range(len(seed_lst)):
    d_new2.append([])
    for j in range(5):
        d_new2[i].append(d_origin[j] if v_bid[i][j]>0 else 0)

# print(d_new2)

(3570.3093264873924, 10122.170673512606)


In [32]:
print(d_new1)
print(d_new2)

[[0, 0, 0, 0, 58], [0, 0, 102, 0, 58], [0, 0, 0, 0, 0], [0, 0, 102, 0, 58], [0, 0, 0, 0, 0], [0, 0, 102, 0, 58], [0, 0, 0, 0, 58], [0, 0, 102, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 102, 0, 0], [0, 0, 102, 0, 58], [0, 0, 0, 0, 0], [0, 0, 0, 0, 58], [0, 0, 102, 0, 0], [0, 0, 0, 0, 58], [0, 0, 0, 0, 58], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 102, 0, 58], [0, 0, 102, 0, 58], [0, 0, 102, 0, 58], [0, 0, 102, 0, 0], [0, 0, 102, 0, 58], [0, 0, 102, 0, 58]]
[[0, 0, 0, 0, 0], [0, 0, 102, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 102, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 102, 0, 0], [0, 0, 102, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 102, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 102, 0, 0], [0, 0, 102, 0, 0], [0, 0, 102, 0, 0], [0, 0, 102, 0, 0], [0, 0, 102, 0, 0], [0, 0, 102, 0, 0]]


## Part (b) Economy and Economy Plus classes in the second period {30,0} prior to departure.
### 1. Getting dual solution of DLP and PNLP for each replication.

In [33]:
import numpy as np
import pandas as pd
import itertools
from pyomo.environ import *
import pyomo.environ as pyo
from pyomo.core import *
from pyomo.pysp.annotations import (PySP_ConstraintStageAnnotation, StochasticConstraintBoundsAnnotation, 
                                    StochasticConstraintBodyAnnotation)

model = ConcreteModel()

# --------k is the replication number-------
k = 0
# ----------k is [0,1, ..., 24]-------------

v = {1:280,2:300,3:190,4:220,5:140}
v_bid1 = [[339, 317, 209, 176, 0], [271, 365, 0, 284, 0], [215, 256, 207, 255, 167], [322, 302, 0, 210, 0], [308, 229, 214, 180, 168],
          [295, 328, 0, 227, 0], [283, 284, 200, 236, 0], [315, 376, 0, 205, 146], [317, 279, 198, 277, 166], [233, 350, 205, 233, 162], 
          [357, 345, 0, 180, 140], [303, 383, 0, 209, 0], [334, 316, 242, 233, 178], [328, 272, 223, 215, 0], [335, 319, 0, 224, 168], 
          [295, 225, 228, 229, 0], [238, 229, 202, 220, 0], [285, 266, 211, 263, 162], [349, 290, 224, 235, 153], [197, 215, 0, 186, 0], 
          [311, 244, 0, 250, 0], [303, 298, 0, 225, 0], [335, 319, 0, 224, 168], [259, 377, 0, 178, 0], [273, 353, 0, 205, 0]]
v_bid2 = [[339, 317, 209, 176, 133], [271, 365, 139, 284, 101], [215, 256, 207, 255, 167], [322, 302, 187, 210, 120], [308, 229, 214, 180, 168],
          [295, 328, 149, 227, 106], [283, 284, 200, 236, 99], [315, 376, 187, 205, 146], [317, 279, 198, 277, 166], [233, 350, 205, 233, 162],
          [357, 345, 133, 180, 140], [303, 383, 144, 209, 138], [334, 316, 242, 233, 178], [328, 272, 223, 215, 121], [335, 319, 166, 224, 168], 
          [295, 225, 228, 229, 120], [238, 229, 202, 220, 106], [285, 266, 211, 263, 162], [349, 290, 224, 235, 153], [197, 215, 155, 186, 101], 
          [311, 244, 173, 250, 122], [303, 298, 145, 225, 126], [335, 319, 166, 224, 168], [259, 377, 169, 178, 121], [273, 353, 139, 205, 107]]
c = [[0,1,1,0,0,1],
     [1,0,1,1,0,0],
     [0,1,1,0,0,1],
     [1,0,1,1,0,0],
     [0,0,1,0,1,0]]
capa = {1:100,2:100,3:300,4:100,5:100,6:100}

d_mean = [50,30,200,100,160]
d_new1 = [[0, 0, 102, 26, 0], [0, 0, 0, 26, 0], [0, 0, 102, 26, 58], [0, 0, 0, 26, 0], [0, 0, 102, 26, 58],
          [0, 0, 0, 26, 0], [0, 0, 102, 26, 0], [0, 0, 0, 26, 58], [0, 0, 102, 26, 58], [0, 0, 102, 26, 58],
          [0, 0, 0, 26, 58], [0, 0, 0, 26, 0], [0, 0, 102, 26, 58], [0, 0, 102, 26, 0], [0, 0, 0, 26, 58],
          [0, 0, 102, 26, 0], [0, 0, 102, 26, 0], [0, 0, 102, 26, 58], [0, 0, 102, 26, 58], [0, 0, 0, 26, 0],
          [0, 0, 0, 26, 0], [0, 0, 0, 26, 0], [0, 0, 0, 26, 58], [0, 0, 0, 26, 0], [0, 0, 0, 26, 0]]
d_new2 = [[0, 0, 0, 0, 0], [0, 0, 102, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 
          [0, 0, 102, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], 
          [0, 0, 102, 0, 0], [0, 0, 102, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 102, 0, 0], 
          [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 102, 0, 0], 
          [0, 0, 102, 0, 0], [0, 0, 102, 0, 0], [0, 0, 102, 0, 0], [0, 0, 102, 0, 0], [0, 0, 102, 0, 0]]

# Possible Demands 
d1_rhs_table = [ceil((d_mean[0]-d_new1[k][0])*0.9),floor((d_mean[0]-d_new1[k][0])*1.1)+1]
d2_rhs_table = [ceil((d_mean[1]-d_new1[k][1])*0.9),floor((d_mean[1]-d_new1[k][1])*1.1)+1]
d3_rhs_table = [ceil((d_mean[2]-d_new1[k][2])*0.9),floor((d_mean[2]-d_new1[k][2])*1.1)+1]
d4_rhs_table = [ceil((d_mean[3]-d_new1[k][3])*0.9),floor((d_mean[3]-d_new1[k][3])*1.1)+1]
d5_rhs_table = [ceil((d_mean[4]-d_new1[k][4])*0.9),floor((d_mean[4]-d_new1[k][4])*1.1)+1]

model.constraint_stage = PySP_ConstraintStageAnnotation()
model.stoch_rhs = StochasticConstraintBoundsAnnotation()
num_scenarios = len(d1_rhs_table) * len(d2_rhs_table) * len(d3_rhs_table) * len(d4_rhs_table) * len(d5_rhs_table)
scenario_data = dict(('Scenario'+str(i), (d1val, d22val, d3val, d4val, d5val))
                     for i, (d1val, d22val, d3val, d4val, d5val) in
                     enumerate (itertools.product(d1_rhs_table, d2_rhs_table, d3_rhs_table, d4_rhs_table, d5_rhs_table),1))

# Declare Variables
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT_EXPORT)

model.x1 = Var(within=NonNegativeIntegers)
model.x2 = Var(within=NonNegativeIntegers)
model.x3 = Var(within=NonNegativeIntegers)
model.x4 = Var(within=NonNegativeIntegers)
model.x5 = Var(within=NonNegativeIntegers)

model.t1 = Var(within=NonNegativeIntegers)
model.t2 = Var(within=NonNegativeIntegers)
model.t3 = Var(within=NonNegativeIntegers)
model.t4 = Var(within=NonNegativeIntegers)
model.t5 = Var(within=NonNegativeIntegers)

model.d1_rhs = Param(mutable=True, initialize=0.0)
model.d2_rhs = Param(mutable=True, initialize=0.0)
model.d3_rhs = Param(mutable=True, initialize=0.0)
model.d4_rhs = Param(mutable=True, initialize=0.0)
model.d5_rhs = Param(mutable=True, initialize=0.0)

# Objective 
model.FirstStageCost = Expression(initialize=0)
model.SecondStageCost = Expression(initialize=model.t1*v_bid1[k][0]+model.t2*v_bid1[k][1]+model.t3*v_bid1[k][2]
                      +model.t4*v_bid1[k][3]+model.t5*v_bid1[k][4])

# Capacity Constraints
c1 =np.transpose(c)[0]
model.c1 = Constraint(expr= model.x1*c1[0]+model.x2*c1[1]+model.x3*c1[2]
                      +model.x4*c1[3]+model.x5*c1[4] <= capa[1])
model.constraint_stage.declare(model.c1,1)
c2 =np.transpose(c)[1]
model.c2 = Constraint(expr= model.x1*c2[0]+model.x2*c2[1]+model.x3*c2[2]
                      +model.x4*c2[3]+model.x5*c2[4] <= capa[2])
model.constraint_stage.declare(model.c2,1)
c3 =np.transpose(c)[2]
model.c3 = Constraint(expr= model.x1*c3[0]+model.x2*c3[1]+model.x3*c3[2]
                      +model.x4*c3[3]+model.x5*c3[4] <= capa[3])
model.constraint_stage.declare(model.c3,1)
c4 =np.transpose(c)[3]
model.c4 = Constraint(expr= model.x1*c4[0]+model.x2*c4[1]+model.x3*c4[2]
                      +model.x4*c4[3]+model.x5*c4[4] <= capa[4])
model.constraint_stage.declare(model.c4,1)
c5 =np.transpose(c)[4]
model.c5 = Constraint(expr= model.x1*c5[0]+model.x2*c5[1]+model.x3*c5[2]
                      +model.x4*c5[3]+model.x5*c5[4] <= capa[5])
model.constraint_stage.declare(model.c5,1)
c6 =np.transpose(c)[5]
model.c6 = Constraint(expr= model.x1*c6[0]+model.x2*c6[1]+model.x3*c6[2]
                      +model.x4*c6[3]+model.x5*c6[4] <= capa[6])
model.constraint_stage.declare(model.c6,1)

# Second Stage Min{X,D} Constraint (when D is stochastic)
model.d11 = Constraint(expr=model.t1 <= model.d1_rhs)
model.constraint_stage.declare(model.d11,2)
model.stoch_rhs.declare(model.d11)

model.d12 = Constraint(expr=model.t1 - model.x1 <= 0)
model.constraint_stage.declare(model.d12,2)

model.d21 = Constraint(expr=model.t2 <= model.d2_rhs)
model.constraint_stage.declare(model.d21,2)
model.stoch_rhs.declare(model.d21)

model.d22 = Constraint(expr=model.t2 - model.x2 <= 0)
model.constraint_stage.declare(model.d22,2)

model.d31 = Constraint(expr=model.t3 <= model.d3_rhs)
model.constraint_stage.declare(model.d31,2)
model.stoch_rhs.declare(model.d31)

model.d32 = Constraint(expr=model.t3 - model.x3 <= 0)
model.constraint_stage.declare(model.d32,2)

model.d41 = Constraint(expr=model.t4 <= model.d4_rhs)
model.constraint_stage.declare(model.d41,2)
model.stoch_rhs.declare(model.d41)

model.d42 = Constraint(expr=model.t4 - model.x4 <= 0)
model.constraint_stage.declare(model.d42,2)

model.d51 = Constraint(expr=model.t5 <= model.d5_rhs)
model.constraint_stage.declare(model.d51,2)
model.stoch_rhs.declare(model.d51)

model.d52 = Constraint(expr=model.t5 - model.x5 <= 0)
model.constraint_stage.declare(model.d52,2)

# Final Objective
model.obj = Objective(expr=(model.FirstStageCost+model.SecondStageCost)*(-1))

def pysp_scenario_tree_model_callback():
    from pyomo.pysp.scenariotree.tree_structure_model import \
        CreateConcreteTwoStageScenarioTreeModel

    st_model = CreateConcreteTwoStageScenarioTreeModel(num_scenarios)

    first_stage = st_model.Stages.first()
    second_stage = st_model.Stages.last()

    # First Stage
    st_model.StageCost[first_stage] = 'FirstStageCost'  
    st_model.StageVariables[first_stage].add('x1')
    st_model.StageVariables[first_stage].add('x2')
    st_model.StageVariables[first_stage].add('x3')
    st_model.StageVariables[first_stage].add('x4')
    st_model.StageVariables[first_stage].add('x5')
    # Second Stage
    st_model.StageCost[second_stage] = 'SecondStageCost'
    st_model.StageVariables[second_stage].add('t1')
    st_model.StageVariables[second_stage].add('t2')
    st_model.StageVariables[second_stage].add('t3')
    st_model.StageVariables[second_stage].add('t4')
    st_model.StageVariables[second_stage].add('t5')
    return st_model

def pysp_instance_creation_callback(scenario_name, node_names):

    instance = model.clone()

    d3_rhs_val, d4_rhs_val, d5_rhs_val  = scenario_data[scenario_name]
    instance.d1_rhs.value = d1_rhs_val
    instance.d2_rhs.value = d2_rhs_val
    instance.d3_rhs.value = d3_rhs_val
    instance.d4_rhs.value = d4_rhs_val
    instance.d5_rhs.value = d5_rhs_val

    return instance

    (constraint stages are automatically inferred). It will be removed in the
    future. Please update your model file.


In [34]:
!python -m pyomo.pysp.convert.smps -m /Users/yul/Desktop/itinerary_network_2periods_py/itinerary_network_2periods_first.py --basename Itinerary1.24 \--output-directory /Users/yul/Desktop/itinerary_network_2periods_py/sdinput/Itinerary1.24 --symbolic-solver-labels

/anaconda3/envs/yulenv/lib/python3.6/runpy.py:125: RuntimeWarning: 'pyomo.pysp.convert.smps' found in sys.modules after import of package 'pyomo.pysp.convert', but prior to execution of 'pyomo.pysp.convert.smps'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
    (constraint stages are automatically inferred). It will be removed in the
    future. Please update your model file.

Total execution time=2.01 seconds


In [35]:
# --------k is the replication number-------
k = 24
# ----------k is [0,1, ..., 24]-------------

d_table1 = []   
for i in range(25):
    d_table1.append([])
    for j in range(5):
        d_table1[i].append(str(ceil((d_mean[j]-d_new1[i][j])*0.9))+' ~ '+str(floor((d_mean[j]-d_new1[i][j])*1.1)+1))
print(d_table1[k])

# d_table2 = []   
# for i in range(25):
#     d_table2.append([])
#     for j in range(5):
#         d_table2[i].append(str(ceil((d_mean[j]-d_new2[i][j])*0.9))+' ~ '+str(floor((d_mean[j]-d_new2[i][j])*1.1)+1))
# print(d_table2[k])

['45 ~ 56', '27 ~ 34', '180 ~ 221', '67 ~ 82', '144 ~ 177']


In [105]:
!python -m pyomo.pysp.convert.smps -m /Users/yul/Desktop/itinerary_network_2periods_py/itinerary_network_2periods_second.py --basename Itinerary2.24 \--output-directory /Users/yul/Desktop/itinerary_network_2periods_py/sdinput/Itinerary2.24 --symbolic-solver-labels

/anaconda3/envs/yulenv/lib/python3.6/runpy.py:125: RuntimeWarning: 'pyomo.pysp.convert.smps' found in sys.modules after import of package 'pyomo.pysp.convert', but prior to execution of 'pyomo.pysp.convert.smps'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
    (constraint stages are automatically inferred). It will be removed in the
    future. Please update your model file.

Total execution time=2.06 seconds


In [106]:
# --------k is the replication number-------
k = 24
# ----------k is [0,1, ..., 24]-------------

# d_table1 = []   
# for i in range(25):
#     d_table1.append([])
#     for j in range(5):
#         d_table1[i].append(str(ceil((d_mean[j]-d_new1[i][j])*0.9))+' ~ '+str(floor((d_mean[j]-d_new1[i][j])*1.1)+1))
# print(d_table1[k])

d_table2 = []   
for i in range(25):
    d_table2.append([])
    for j in range(5):
        d_table2[i].append(str(ceil((d_mean[j]-d_new2[i][j])*0.9))+' ~ '+str(floor((d_mean[j]-d_new2[i][j])*1.1)+1))
print(d_table2[k])

['45 ~ 56', '27 ~ 34', '89 ~ 108', '90 ~ 111', '144 ~ 177']


In [36]:
# list of dual solutions
dual_sol1 = [[0,0,0,0,0,208.51], [0,0,0,0,0,0], [0,0,0,0,166.39,206,24], [0,0,0,0,166.39,206,24], [0,0,0,0,0,206.24],
            [0,0,0,0,166.98,206.97], [0,199.78,0,0,0,0], [0,0,0,0,0,0], [0,0,0,0,0,197.53], [0,205.00,0,0,0,0],
            [0,0,0,0,0,0], [0,0,0,0,0,0], [0,241.75,0,0,0,0], [0,0,0,0,0,222.43], [0,0,0,0,0,0],
            [0,0,0,0,0,227.80], [0,0,0,0,0,201.35], [0,0,0,0,0,210.40], [0,0,0,0,0,223.46], [0,0,0,0,0,0],
            [0,0,0,0,0,0], [0,0,0,0,0,0], [0,0,0,0,0,0], [0,0,0,0,0,0], [0,0,0,0,0,0]]
dual_sol2 = [[0,0,63.48,112.10,69.20,145.02], [0,0,51.85,231.31,48.85,86.74], [0,0,76.77,177.19,89.55,129.39], [0,0,57.11,152.34,62.57,129.40], [0,0,74.94,104.53,92.56,138.43],
             [0,0,55.20,171.21,50.52,93.41], [0,0,55.20,179.82,43.39,143.97], [0,0,66.19,138.33,79.47,120.38], [0,0,76.49,198.67,88.41,120.20], [0,0,76.20,156.01,85.25,128.11],
             [0,0,54.18,125.27,85.40,78.43], [0,0,61.21,147.34,76.49,82.48], [0,0,65.44,143.12,72.27,78.55], [0,0,64.50,149.86,56.14,157.84], [0,0,77.71,145.70,89.85,87.85],
             [0,0,60.59,163.20,58.77,166.19], [0,0,55.50,163.50,50.02,145.58], [0,0,78.65,182.85,82.42,131.15], [0,0,74.37,159.91,78.16,148.95], [0,0,44.43,140.84,56.17,109.96],
             [0,0,54.42,187.88,66.73,117.38], [0,0,58.39,165.97,67.25,86.20], [0,0,77.71,145.70,89.85,87.85], [0,0,56.33,121.22,64.37,112.25], [0,0,52.46,152.09,54.30,86.23]]

### 2. Getting dual price of itinerary for each replication.

In [37]:
# Dual price for itinerary
dual_price1 = []
for i in range(len(seed_lst)):
    dual_price1.append([0,0,0,0,0])
    
for i in range(len(seed_lst)):
    dual_price1[i][0] = dual_sol1[i][1]+dual_sol1[i][2]+dual_sol1[i][5]
    dual_price1[i][1] = dual_sol1[i][0]+dual_sol1[i][2]+dual_sol1[i][3]
    dual_price1[i][2] = dual_sol1[i][1]+dual_sol1[i][2]+dual_sol1[i][5]
    dual_price1[i][3] = dual_sol1[i][0]+dual_sol1[i][2]+dual_sol1[i][3]
    dual_price1[i][4] = dual_sol1[i][2]+dual_sol1[i][4]

dual_price2 = []
for i in range(len(seed_lst)):
    dual_price2.append([0,0,0,0,0])
    
for i in range(len(seed_lst)):
    dual_price2[i][0] = dual_sol2[i][1]+dual_sol2[i][2]+dual_sol2[i][5]
    dual_price2[i][1] = dual_sol2[i][0]+dual_sol2[i][2]+dual_sol2[i][3]
    dual_price2[i][2] = dual_sol2[i][1]+dual_sol2[i][2]+dual_sol2[i][5]
    dual_price2[i][3] = dual_sol2[i][0]+dual_sol2[i][2]+dual_sol2[i][3]
    dual_price2[i][4] = dual_sol2[i][2]+dual_sol2[i][4]
# print(dual_price2)

# Demand for period 2 only (take-out the used-up demand in period 1).
d_period2_1 = []
for i in range(len(seed_lst)):
    d_period2_1.append([])
    
for i in range(len(seed_lst)):
    np.random.seed(seed_lst[i])
    for j in range(5):
        d_period2_1[i].append(np.random.randint(low=ceil((d_mean[j]-d_new1[i][j])*0.9),
                                              high=floor((d_mean[j]-d_new1[i][j])*1.1)+1))
# print(d_period2_1)

d_period2_2 = []
for i in range(len(seed_lst)):
    d_period2_2.append([])
    
for i in range(len(seed_lst)):
    np.random.seed(seed_lst[i])
    for j in range(5):
        d_period2_2[i].append(np.random.randint(low=ceil((d_mean[j]-d_new2[i][j])*0.9),
                                              high=floor((d_mean[j]-d_new2[i][j])*1.1)+1))
# print(d_period2_2)

### 3. Generate 25 replications with random seeds deriven from seed(1) and calculate the expected total revenue with respect to the dual price for each itinerary.
#### The customer bids of period 2 is driven from the uniform distribution $ \{v_i-0.1v_i,  v_i+0.1v_i\} $
#### 1. DLP :  Calculate the 95% Confidence Interval for expected total revenue.

In [44]:
np.random.seed(1)
seed_lst= np.random.randint(100,size=25)

v = [280,300,190,220,140]

v_bid = []    
for i in range(len(seed_lst)): # create a list w.r.t. seeds with nested v_i lists
    np.random.seed(seed_lst[i])
    v_bid.append([])
    for j in range(5):
        v_bid[i].append(np.random.randint(low=v[j]*0.9,high=v[j]*1.1)) # fills nested lists with data        
# print(v_bid)

# DLP 
for i in range(len(seed_lst)):
    for j in range(5):
        if v_bid[i][j] >= dual_price1[i][j]:
            v_bid[i][j] = 0
        else: v_bid[i][j]
# print(v_bid)

# Total revenue for 25 replications
total_revenue_1=[]
for i in range(len(seed_lst)):
    total_revenue_1.append(v_bid[i][0]*d_period2_1[i][0]+v_bid[i][1]*d_period2_1[i][1]
                         +v_bid[i][2]*d_period2_1[i][2]+v_bid[i][3]*d_period2_1[i][3]
                         +v_bid[i][4]*d_period2_1[i][4])
# print(total_revenue)

import scipy.stats as st
ci2_1 = st.t.interval(0.95, len(total_revenue_1)-1, loc=np.mean(total_revenue_1),
              scale=st.sem(total_revenue_1))
print(ci2_1)

(7190.245275440534, 21480.074724559465)


#### 2. PNLP :  Calculate the 95% Confidence Interval for expected total revenue.

In [45]:
# PNLP 
for i in range(len(seed_lst)):
    for j in range(5):
        if v_bid[i][j] >= dual_price2[i][j]:
            v_bid[i][j] = 0
        else: v_bid[i][j]
# print(v_bid)

# Total revenue for 25 replications
total_revenue_2=[]
for i in range(len(seed_lst)):
    total_revenue_2.append(v_bid[i][0]*d_period2_2[i][0]+v_bid[i][1]*d_period2_2[i][1]
                         +v_bid[i][2]*d_period2_2[i][2]+v_bid[i][3]*d_period2_2[i][3]
                         +v_bid[i][4]*d_period2_2[i][4])
# print(total_revenue)

import scipy.stats as st
ci2_2 = st.t.interval(0.95, len(total_revenue_2)-1, loc=np.mean(total_revenue_2),
              scale=st.sem(total_revenue_2))
print(ci2_2)

(8670.8071419958796, 25169.912858004122)
